## Portfolio Optimization using PCA 

In [1]:
# Import relevant libraries

import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
import yfinance as yf 
from sklearn.decomposition import PCA 
from sklearn.preprocessing import StandardScaler 

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
#Fetch list of company trackers of S&P 500

source = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
stock_name = pd.read_html(source)[0]

symbols = stock_name.Symbol.to_list()
symbols

['MMM',
 'AOS',
 'ABT',
 'ABBV',
 'ACN',
 'ADBE',
 'AMD',
 'AES',
 'AFL',
 'A',
 'APD',
 'ABNB',
 'AKAM',
 'ALB',
 'ARE',
 'ALGN',
 'ALLE',
 'LNT',
 'ALL',
 'GOOGL',
 'GOOG',
 'MO',
 'AMZN',
 'AMCR',
 'AEE',
 'AEP',
 'AXP',
 'AIG',
 'AMT',
 'AWK',
 'AMP',
 'AME',
 'AMGN',
 'APH',
 'ADI',
 'ANSS',
 'AON',
 'APA',
 'APO',
 'AAPL',
 'AMAT',
 'APTV',
 'ACGL',
 'ADM',
 'ANET',
 'AJG',
 'AIZ',
 'T',
 'ATO',
 'ADSK',
 'ADP',
 'AZO',
 'AVB',
 'AVY',
 'AXON',
 'BKR',
 'BALL',
 'BAC',
 'BAX',
 'BDX',
 'BRK.B',
 'BBY',
 'TECH',
 'BIIB',
 'BLK',
 'BX',
 'BK',
 'BA',
 'BKNG',
 'BWA',
 'BSX',
 'BMY',
 'AVGO',
 'BR',
 'BRO',
 'BF.B',
 'BLDR',
 'BG',
 'BXP',
 'CHRW',
 'CDNS',
 'CZR',
 'CPT',
 'CPB',
 'COF',
 'CAH',
 'KMX',
 'CCL',
 'CARR',
 'CAT',
 'CBOE',
 'CBRE',
 'CDW',
 'CE',
 'COR',
 'CNC',
 'CNP',
 'CF',
 'CRL',
 'SCHW',
 'CHTR',
 'CVX',
 'CMG',
 'CB',
 'CHD',
 'CI',
 'CINF',
 'CTAS',
 'CSCO',
 'C',
 'CFG',
 'CLX',
 'CME',
 'CMS',
 'KO',
 'CTSH',
 'CL',
 'CMCSA',
 'CAG',
 'COP',
 'ED',
 'STZ',
 

In [3]:
# Define the date range
start_date = "2020-01-01"
end_date = "2024-12-31"

# Fetch the benchmark for trading days (e.g., S&P 500 index or a known stock like SPY)
benchmark = yf.download("SPY", start=start_date, end=end_date)

# Use the benchmark's dates as the reference trading calendar
trading_days = benchmark.index

# Initialize a dictionary to store valid tickers and their data
valid_tickers = {}

# Iterate over tickers to pull data
for symbol in symbols:
    try:
        # Download historical data
        data = yf.download(symbol, start=start_date, end=end_date)
        
        # Ensure the data has no missing dates compared to the trading calendar
        if not data.empty:
            # Align data to the trading calendar
            aligned_data = data.reindex(trading_days)
            
            # Check for missing rows or NaN values
            if aligned_data.isna().sum().sum() == 0:
                valid_tickers[symbol] = aligned_data
    except:
        # Skip tickers with issues
        pass

# Combine all valid data into a single DataFrame
combined_data = pd.concat(valid_tickers, axis=1)

print(f"Valid tickers with complete data: {list(valid_tickers.keys())}")
print('Dataset shape: ', combined_data.shape)
combined_data.head()


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

Valid tickers with complete data: ['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A', 'APD', 'AKAM', 'ALB', 'ARE', 'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AEE', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'AON', 'APA', 'APO', 'AAPL', 'AMAT', 'APTV', 'ACGL', 'ADM', 'ANET', 'AJG', 'AIZ', 'T', 'ATO', 'ADSK', 'ADP', 'AZO', 'AVB', 'AVY', 'AXON', 'BKR', 'BALL', 'BAC', 'BAX', 'BDX', 'BBY', 'TECH', 'BIIB', 'BLK', 'BX', 'BK', 'BA', 'BKNG', 'BWA', 'BSX', 'BMY', 'AVGO', 'BR', 'BRO', 'BLDR', 'BG', 'BXP', 'CHRW', 'CDNS', 'CZR', 'CPT', 'CPB', 'COF', 'CAH', 'KMX', 'CCL', 'CAT', 'CBOE', 'CBRE', 'CDW', 'CE', 'COR', 'CNC', 'CNP', 'CF', 'CRL', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'CL', 'CMCSA', 'CAG', 'COP', 'ED', 'STZ', 'COO', 'CPRT', 'GLW', 'CPAY', 'CTVA', 'CSGP', 'COST', 'CTRA', 'CRWD', 'CCI', 'CSX', 'CMI', 'CVS', 'DHR', 'DRI', 'DVA', 'D

MMM                                                  \
                  Open        High         Low       Close   Adj Close   
Date                                                                     
2020-01-02  148.561874  150.510040  148.110367  150.501678  123.614937   
2020-01-03  148.010040  149.381271  146.847824  149.205688  122.550491   
2020-01-06  148.118729  149.423080  147.449829  149.347824  122.667221   
2020-01-07  149.063538  149.255859  147.842804  148.745819  122.172768   
2020-01-08  148.829437  151.755859  148.536789  151.028427  124.047592   

                           AOS                                   ...  \
             Volume       Open       High        Low      Close  ...   
Date                                                             ...   
2020-01-02  4307633  47.910000  47.980000  47.209999  47.770000  ...   
2020-01-03  2950412  47.160000  47.500000  46.750000  47.349998  ...   
2020-01-06  2389608  47.130001  47.669998  46.759998  47.650002  ...   
2020-01-07  2598908  47.400002  47.740002  47.139999  47.330002  ...   
2020-01-08  3298927  47.419998  47.810001  47.189999  47.259998  ...   

                   ZBH                                          ZTS  \
                   Low       Close   Adj Close   Volume        Open   
Date                                                                  
2020-01-02  143.174759  144.854370  139.626221  1083972  132.050003   
2020-01-03  141.708740  144.475723  139.261246   992405  132.479996   
2020-01-06  142.941742  143.640778  138.456421   972423  133.779999   
2020-01-07  142.417480  143.514557  138.334793   934519  133.000000   
2020-01-08  143.145630  145.233017  139.991196   936064  133.759995   

                                                                     
                  High         Low       Close   Adj Close   Volume  
Date                                                                 
2020-01-02  134.279999  131.479996  134.139999  129.208878  1576700  
2020-01-03  134.910004  132.270004  134.160004  129.228104  1274000  
2020-01-06  134.070007  132.710007  133.130005  128.236008  2334100  
2020-01-07  134.809998  132.669998  133.580002  128.669479  1224500  
2020-01-08  135.270004  133.240005  133.289993  128.390121  1766700  

[5 rows x 2940 columns]

In [47]:
# Extract only the 'Adj Close' prices for all tickers
adj_close_data = combined_data.xs('Adj Close', axis=1, level=1)
adj_close_data.head()


,MMM,AOS,ABT,ABBV,ACN,ADBE,AMD,AES,AFL,A,...,WY,WMB,WTW,WYNN,XEL,XYL,YUM,ZBRA,ZBH,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2020-01-02,123.614952,43.642891,79.689949,72.264877,195.263580,334.429993,49.099998,17.096725,47.223614,83.214867,...,24.381449,17.491032,191.207962,140.138123,54.000294,75.240524,93.090340,259.140015,139.626205,129.208862
2020-01-03,122.550491,43.259171,78.718460,71.578949,194.938370,331.809998,48.599998,16.900408,46.896107,81.878799,...,24.422930,17.565496,191.254776,138.059448,54.259987,75.683395,92.798767,256.049988,139.261230,129.228149
2020-01-06,122.667221,43.533260,79.130875,72.143845,193.665405,333.709991,48.389999,17.096725,46.763329,82.120842,...,24.373156,17.796322,191.488785,137.786194,54.182079,75.193413,92.744110,258.010010,138.456421,128.235992
2020-01-07,122.172783,43.240910,78.690956,71.732269,189.484207,333.390015,48.250000,17.156477,46.320751,82.372551,...,24.215538,17.796322,191.095642,138.440063,54.069538,74.910728,92.908119,256.470001,138.334778,128.669479
2020-01-08,124.047607,43.176952,79.011734,72.240669,189.855865,337.869995,47.830002,17.122330,46.462379,83.185837,...,24.306782,17.520819,191.310944,139.308609,54.017609,75.165138,93.072113,247.639999,139.991226,128.390152


In [52]:
final_df = adj_close_data.copy()

stock_returns = final_df.pct_change().dropna()

stock_returns.head()

,MMM,AOS,ABT,ABBV,ACN,ADBE,AMD,AES,AFL,A,...,WY,WMB,WTW,WYNN,XEL,XYL,YUM,ZBRA,ZBH,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2020-01-03,-0.008611,-0.008792,-0.012191,-0.009492,-0.001665,-0.007834,-0.010183,-0.011483,-0.006935,-0.016056,...,0.001701,0.004257,0.000245,-0.014833,0.004809,0.005886,-0.003132,-0.011924,-0.002614,0.000149
2020-01-06,0.000953,0.006336,0.005239,0.007892,-0.006530,0.005726,-0.004321,0.011616,-0.002831,0.002956,...,-0.002038,0.013141,0.001224,-0.001979,-0.001436,-0.006474,-0.000589,0.007655,-0.005779,-0.007678
2020-01-07,-0.004031,-0.006716,-0.005559,-0.005705,-0.021590,-0.000959,-0.002893,0.003495,-0.009464,0.003065,...,-0.006467,0.000000,-0.002053,0.004746,-0.002077,-0.003759,0.001768,-0.005969,-0.000879,0.003380
2020-01-08,0.015346,-0.001479,0.004076,0.007087,0.001961,0.013438,-0.008705,-0.001990,0.003058,0.009873,...,0.003768,-0.015481,0.001127,0.006274,-0.000960,0.003396,0.001765,-0.034429,0.011974,-0.002171
2020-01-09,0.003156,-0.004443,0.002668,0.007708,0.008907,0.007636,0.023834,0.009970,0.000190,0.015712,...,0.008192,0.005950,0.002985,-0.004203,0.002243,0.004513,0.007929,-0.004603,-0.006618,0.013279


In [53]:
norm_returns = (stock_returns - stock_returns.mean())/stock_returns.std()
norm_returns.head()

,MMM,AOS,ABT,ABBV,ACN,ADBE,AMD,AES,AFL,A,...,WY,WMB,WTW,WYNN,XEL,XYL,YUM,ZBRA,ZBH,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2020-01-03,-0.473191,-0.489518,-0.764582,-0.660329,-0.126050,-0.340992,-0.345675,-0.456702,-0.382208,-0.887107,...,0.052995,0.139994,-0.017924,-0.442566,0.278703,0.272019,-0.216032,-0.496771,-0.131285,-0.011673
2020-01-06,0.040010,0.303681,0.291862,0.451424,-0.388052,0.211428,-0.168512,0.453595,-0.180112,0.127975,...,-0.099242,0.539439,0.038484,-0.064503,-0.107997,-0.357703,-0.061086,0.276369,-0.291290,-0.439011
2020-01-07,-0.227400,-0.380633,-0.362642,-0.418141,-1.199156,-0.060904,-0.125362,0.133548,-0.506745,0.133795,...,-0.279550,-0.051434,-0.150364,0.133290,-0.147706,-0.219396,0.082541,-0.261604,-0.043559,0.164742
2020-01-08,0.812374,-0.106076,0.221391,0.399976,0.069293,0.525579,-0.300989,-0.082618,0.109883,0.497299,...,0.137133,-0.747528,0.032901,0.178240,-0.078559,0.145165,0.082341,-1.385446,0.606166,-0.138352
2020-01-09,0.158234,-0.261500,0.136023,0.439655,0.443370,0.289237,0.682362,0.388731,-0.031305,0.809052,...,0.317226,0.216098,0.139977,-0.129918,0.119832,0.202063,0.457908,-0.207689,-0.333714,0.705211


In [56]:
train_cutoff = '2024-01-01'

df_train = norm_returns[norm_returns.index <= train_cutoff]
df_test = norm_returns[norm_returns.index > train_cutoff]


print('train dataset shape: ', df_train.shape)
print('test dataset shape: ', df_test.shape)

train dataset shape:  (1005, 488)
test dataset shape:  (246, 488)


In [57]:
pca = PCA()

principal_components = pca.fit_transform(df_train)

explained_variance_ratio = pca.explained_variance_ratio_
cumulative_variance = np.cumsum(explained_variance_ratio)

In [63]:
print(explained_variance_ratio)

[4.39746701e-01 5.85706723e-02 4.66257149e-02 1.88479043e-02
 1.60735970e-02 1.21525130e-02 1.12417286e-02 9.78090631e-03
 8.49080003e-03 8.34498939e-03 7.41505477e-03 6.73963456e-03
 6.33559699e-03 6.23970127e-03 5.90619273e-03 5.16634828e-03
 5.01263510e-03 4.85663492e-03 4.60776929e-03 4.41167705e-03
 4.21542878e-03 4.13555560e-03 4.01558197e-03 3.73140115e-03
 3.63972663e-03 3.44911164e-03 3.35995626e-03 3.33311517e-03
 3.29678913e-03 3.11305074e-03 3.08204270e-03 2.88733354e-03
 2.78609571e-03 2.73310699e-03 2.68685769e-03 2.57923381e-03
 2.53650408e-03 2.47678399e-03 2.38136467e-03 2.37257576e-03
 2.31182022e-03 2.28658822e-03 2.27509281e-03 2.24691248e-03
 2.23754337e-03 2.20418210e-03 2.19730016e-03 2.16845826e-03
 2.12063047e-03 2.08124267e-03 2.04621269e-03 2.03152291e-03
 2.01972050e-03 1.99482089e-03 1.96716989e-03 1.94131268e-03
 1.90413281e-03 1.88612484e-03 1.87229717e-03 1.83721315e-03
 1.82676120e-03 1.81376951e-03 1.79073258e-03 1.75750360e-03
 1.72886612e-03 1.709085